## 1 高阶函数

In [8]:
# 函数定义与调用
def f(x, y):
    print("function: f(x, y)......")
    return (x + y) * 2

print(f)
print(f(1, 2))
print("----------------")

# 函数作为值进行传递
f1 = f
print(f1)
print(f1(1, 2))
print("----------------")

# 函数作为参数进行传递
def f3(x, y, func):
    print("function: f3(x, y, func)......")
    return func(x, y)

print(f3)
print(f3(1, 2, f))
print("----------------")

# 函数作为函数返回值返回
def f4():
    def func():
        return 1
    return func

print(f4)
print(f4())
print(f4()())
# print(f4()()) ==>
t = f4()
print(t())

<function f at 0x000001FDF3525A60>
function: f(x, y)......
6
----------------
<function f at 0x000001FDF3525A60>
function: f(x, y)......
6
----------------
<function f3 at 0x000001FDF3522550>
function: f3(x, y, func)......
function: f(x, y)......
6
----------------
<function f4 at 0x000001FDF3522C10>
<function f4.<locals>.func at 0x000001FDF3522D30>
1
1


In [15]:
from functools import reduce

# 一些高阶函数
# map
def f5(x):
    return x * x

m = map(f5, [1, 2, 3])
print(list(m))
print("----------------")

# reduce
def f6(x ,y):
    return x + y
r = reduce(f6, [1, 2, 3])
print(r)
print("----------------")

# filter
def f7(x):
    return x % 2 == 1
fi = filter(f7, [1, 2, 3])
print(list(fi))
print("----------------")

# sorted
s = sorted([4, -2, 1, -6], key=abs)
print(s)

d = [('c',3), ('a',1), ('d',4), ('b',2)]
def f8(x):
    return x[1]
ss = sorted(d, key=f8)
print(ss)

[1, 4, 9]
----------------
6
----------------
[1, 3]
----------------
[1, -2, 4, -6]
[('a', 1), ('b', 2), ('c', 3), ('d', 4)]


## 2 闭包
如果一个函数，访问到了它的外部（局部）变量的值，那么这个函数和他所处的环境，称为闭包。

参考：[https://www.liaoxuefeng.com/wiki/1016959663602400/1017434209254976](https://www.liaoxuefeng.com/wiki/1016959663602400/1017434209254976)

In [19]:
def lazy_sum(*args):
    def add():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return add

ls = lazy_sum(1, 2, 3)
print(lazy_sum)
# `ls = lazy_sum(1, 2, 3)` 返回的是 add 函数，此时并不会被立即执行计算结果，而是执行 `ls()` 调用这个函数时才能输出结果
print(ls)
print(ls())

# 内部的 add 函数引用了其外部的 args 变量，
# 当 lazy_sum 执行完成，返回 add 函数时，args 变量保存在返回的 add 函数中
# （应该随着 lazy_sum 执行完成，释放 args 变量占用的空间，但它没有释放，保存在了 add 函数中），
# 当调用 add 函数时，就会使用这个 args 变量，输出结果。

<function lazy_sum at 0x000001FDF1990F70>
<function lazy_sum.<locals>.add at 0x000001FDF1990AF0>
6


In [ ]:
# 返回函数不要引用任何循环变量，或者后续会发生变化的变量
def count():
    fs = []
    for i in range(1, 4):
        def f9():
             return i*i
        print("f9...", f9)
        fs.append(f9)
    return fs

f91, f92, f93 = count()
print(f91)
print(f92)
print(f93)
# 以下结果都是9
# 定义的f9函数引用了外部的变量i，
# 执行 `f91, f92, f93 = count()` 后，f9函数并不会被立即执行，只是把定义的这个f9函数放进了一个列表里，而此时，i已变成了3，
# 当调用这三个函数时，函数执行计算结果，结果都成为了9
print(f91())
print(f92())
print(f93())

In [24]:
# 如果一定要引用循环变量怎么办？
# 方法是再创建一个函数，用该函数的参数(j)绑定循环变量当前的值(i)，无论该循环变量后续如何更改，已绑定到函数参数的值不变
# （传给f10的值i，就保存在了f10的局部变量中，此时，函数g引用的外部变量就是f10的局部变量）
def count():
    def f10(j):
        def g():
            return j*j
        return g
    fs = []
    for i in range(1, 4):
        print("f10(i)...", f10(i))
        fs.append(f10(i)) # f10(i)立刻被执行，因此i的当前值被传入f10()
    return fs

f101, f102, f103 = count()
print(f101())
print(f102())
print(f103())

f10(i)... <function count.<locals>.f10.<locals>.g at 0x000001FDF3525430>
f10(i)... <function count.<locals>.f10.<locals>.g at 0x000001FDF1990D30>
f10(i)... <function count.<locals>.f10.<locals>.g at 0x000001FDF1990790>
1
4
9


In [25]:
# 使用闭包，就是内层函数引用了外层函数的局部变量。如果对外层变量赋值，就会存在问题，需要给引用的外部变量使用 nonlocal 修饰。
# 不加上 `nonlocal x` 就表示 x 成为 fn 的局部变量，但没有初始化所以报错，
# 加上 `nonlocal x` 就表示 x 不是 fn 的局部变量，且已被初始化
def inc():
    x = 0
    def fn():
        nonlocal x
        x = x + 1
        return x
    return fn

f = inc()
print(f()) # 1
print(f()) # 2

# 所以，使用闭包时，对外层变量赋值前，需要先使用 nonlocal 声明该变量不是当前函数的局部变量

1
1


## 3 装饰器

[https://www.liaoxuefeng.com/wiki/1016959663602400/1017451662295584](https://www.liaoxuefeng.com/wiki/1016959663602400/1017451662295584)

In [33]:
# 使用装饰器可以运行些预备代码，也可以在运行后执行一些清理工作。

# 定义一个装饰器函数，它是一个高阶函数，返回值是一个函数，形成闭包。
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

"""
用log装饰器修饰now()，相当于log(now)，返回 wrapper函数 对象，当使用这个对象时，会执行wrapper函数内的语句.

wrapper()函数的参数定义是`(*args, **kw)`，因此，wrapper()函数可以接受任意参数的调用。

在wrapper()函数内，首先打印日志，再紧接着调用原始函数。
"""
@log
def now():
    print('2015-3-25')

now()
print("-----------------------")
n = log(now)
print(n)
n()

call now():
2015-3-25
-----------------------
<function log.<locals>.wrapper at 0x000001FDF356F9D0>
call wrapper():
call now():
2015-3-25


In [34]:
# 如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

# 这个3层嵌套的decorator用法如下：
@log('execute')
def now():
    print('2015-3-25')
now()
print("-----------------------")

# 和两层嵌套的decorator相比，3层嵌套的效果是这样的：
now = log('execute')(now)
# 我们来剖析上面的语句，首先执行log(‘execute’)，返回的是decorator函数，再调用返回的函数，参数是now函数，返回值最终是wrapper函数。

execute now():
2015-3-25


In [ ]:
# 以上两种decorator的定义都没有问题，但还差最后一步。因为我们讲了函数也是对象，它有__name__等属性，
# 但你去看经过decorator装饰之后的函数，它们的__name__已经从原来的’now’变成了’wrapper’：

print(now.__name__)

# 因为返回的那个wrapper()函数名字就是’wrapper’，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

# 不需要编写wrapper.__name__ = func.__name__这样的代码，Python内置的functools.wraps就是干这个事的，所以，一个完整的decorator的写法如下：

import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper


# 或者针对带参数的decorator：
def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator
